In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

# Đọc dữ liệu từ tệp CSV
try:
    data = pd.read_csv('../../../data/BCP.csv')

    # Xem trước dữ liệu
    print("Dữ liệu gốc:")
    print(data.head())
    print(data.columns)

    # Loại bỏ cột 'Time' vì không sử dụng trong tính toán
    data = data.drop(columns=['Time'])

    # Kiểm tra giá trị NaN
    print("\nKiểm tra giá trị NaN:")
    print(data.isna().sum())

    # Xóa các hàng chứa giá trị NaN
    data.dropna(inplace=True)

    # Giả sử chúng ta sẽ chọn cột '440FI539' làm mục tiêu và các cột còn lại là đặc trưng
    target_column = '440FI539'  # Bạn có thể chọn cột khác nếu cần
    features = [col for col in data.columns if col != target_column]

    # Chuyển đổi cột mục tiêu thành nhị phân (0 hoặc 1) dựa trên giá trị trung bình
    data['target_binary'] = (data[target_column] > data[target_column].mean()).astype(int)

    X = data[features]
    y = data['target_binary']

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Huấn luyện mô hình hồi quy Logistic
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train, y_train)

    # Dự đoán với mô hình hồi quy Logistic
    y_pred_logistic = logistic_model.predict(X_test)

    # Huấn luyện mô hình MLP
    mlp_model = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)
    mlp_model.fit(X_train, y_train)

    # Dự đoán với mô hình MLP
    y_pred_mlp = mlp_model.predict(X_test)

    # Kết hợp kết quả dự đoán từ cả hai mô hình
    y_pred_combined = (y_pred_logistic + y_pred_mlp) / 2
    y_pred_combined = np.round(y_pred_combined).astype(int)

    # Đánh giá độ chính xác và F1 score
    accuracy = accuracy_score(y_test, y_pred_combined)
    f1 = f1_score(y_test, y_pred_combined)

    # Thêm cột dự đoán vào dữ liệu ban đầu với tên cột mới
    X_test_df = pd.DataFrame(X_test, columns=features)
    data.loc[X_test_df.index, f'{target_column}_prediction'] = y_pred_combined

    # Xuất dữ liệu ra tệp CSV mới
    # data = pd.read_csv('../../../data/gdp.csv')
    output_file = '../../../data/BCP_predict.csv'
    data.to_csv(output_file, index=False)

    # Hiển thị kết quả
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print(f"Data Result with Predictions:\n{data.head()}")

except Exception as e:
    print(f"Error processing the data: {e}")


Dữ liệu gốc:
   Year         GDP
0  1985  235.652540
1  1986  430.188501
2  1987  585.304937
3  1988  397.014360
4  1989   96.130362
Index(['Year', 'GDP'], dtype='object')
Error processing the data: "['Time'] not found in axis"
